# The whole process:

In [53]:
#import libraries

#pip install google-api-python-client
import os
import pandas as pd
from googleapiclient.discovery import build
from typing import List
from urllib.parse import urlparse, parse_qs

In [54]:
# ask the user to paste a link to any youtube video
youtube_link = 'https://www.youtube.com/watch?v=v7CQkivQNQI'

In [55]:
# source:
# https://stackoverflow.com/questions/45579306/get-youtube-video-url-or-youtube-video-id-from-a-string-using-regex
def get_id(url):
    # function that extracts youtube video ID from a complete link
    u_pars = urlparse(url)
    quer_v = parse_qs(u_pars.query).get('v')
    if quer_v:
        return quer_v[0].strip()
    pth = u_pars.path.split('/')
    if pth:
        return pth[-1].strip()

In [56]:
# extract video_id
video_id = get_id(youtube_link)

In [57]:
# Get API key and connect to Youtube API
yt_api_key = os.environ.get('YT_API_KEY')
yt_connection = build('youtube', 'v3', developerKey = yt_api_key)

In [58]:
# based on:
# https://www.geeksforgeeks.org/how-to-extract-youtube-comments-using-youtube-api-python/
def video_comments(video_id: str, limit: int = 15000) -> List:
    # function that extracts comments from a video
    # empty list for storing replies
    replies = []
    all_replies = []
    # creating youtube resource object
    yt_connection = build('youtube', 'v3', developerKey = yt_api_key)
 
    # retrieve youtube video results
    video_response=yt_connection.commentThreads().list(
        part='snippet,replies',
        videoId=video_id,
        maxResults = 100 # MAX 100
        ).execute()
    # iterate video response
    while video_response:
        # extracting required info
        # from each result object
        for item in video_response['items']:
            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            all_replies.append(comment)
            # counting number of reply of comment
            replycount = item['snippet']['totalReplyCount']
 
            # if reply is there
            if replycount>0:
               
                # iterate through all reply
                for reply in item['replies']['comments']:
                   
                    # Extract reply
                    reply = reply['snippet']['textDisplay']
                    
                    # Store reply is list
                    replies.append(reply)
                    all_replies.append(reply)
 
            # empty reply list
            replies = []
        if len(all_replies) > limit:
            break
         #Again repeat
        if 'nextPageToken' in video_response:
            video_response = yt_connection.commentThreads().list(
                    part = 'snippet,replies',
                    videoId = video_id,
                    maxResults = 100,
                    pageToken=video_response['nextPageToken']
                    ).execute()
        else:
            break
    return all_replies

In [59]:
# Call function
result = video_comments(video_id)

In [60]:
# get the title of the video
response_title = yt_connection.videos().list(
     part = 'snippet',
     id = video_id
).execute()

video_title = response_title['items'][0]['snippet']['title']

In [61]:
df_comments = pd.DataFrame({"Comment":result})

In [62]:
print("Video title:\n{}\n\nNumber of comments that have been found: \n{}".format(video_title,len(result)))

Video title:
Janusz Korwin-Mikke: Imigranci to zalew Europy śmieciem ludzkim, który nie chce pracować

Number of comments that have been found: 
1223


In [63]:
df_comments.tail(100)

,Comment
1123,"zaorane, lewactwo zniszczone :v"
1124,I te miny po przemówieniu :D
1125,no ciezko sie nie zgodzic..
1126,Czy może ktoś przetłumaczyć co na końcu ta bab...
1127,+John Salomon ktos pisal wczesniej ze ta baba ...
...,...
1218,+jeje691 Po raz pierwszy od dawna usłyszała pr...
1219,"+jeje691 Szok, mózg się jej uruchomił, walczy ..."
1220,"+jeje691 zastanawiam się czy to kobieta, mężcz..."
1221,+jeje691 Dziwisz się? Tylko Pan Korwin ma jaja...


In [64]:
# #SAVE TO .xlsx FILE

# from datetime import date

# today = str(date.today())[5:].replace('-','_')
# print("Today's date:", today)
# # dictionary of lists
# comms = {f'Comments- video title:{video_title}, link:{youtube_link}':result} 
# df_save = pd.DataFrame(comms)
# #path = r'C:\Users\Chill\Desktop\INZYNIERKA\dane\extracted_comments\comms{}.xlsx'.format(today)
# path = r'C:\Users\Chill\Desktop\INZYNIERKA\dane\extracted_comments\imigranci_korwin.xlsx'.format(today)
# # saving the dataframe
# df_save.to_excel(path, index=False, header=True,encoding="utf-8")

Today's date: 10_29
